In [565]:
import os
import zipfile
import pandas as pd
import glob

In [566]:
#Cambiar directorio de trabajo y crear carpeta para guardar los archivos
os.chdir("D:/")
os.makedirs("ine", exist_ok=True)
os.chdir("ine")

In [567]:
#Ver archivos en zip
#PRIMERO SE TIENEN QUE DESCARGAR TODOS LOS ARCHIVOS DE LA PÁGINA: https://ime.gob.mx/estadisticas
lista_zip = os.listdir()
for archivo in lista_zip:
    if archivo.endswith(".zip"):
        with zipfile.ZipFile(archivo) as zf:
            zf.extractall()

#Revisar archivos en directorio
os.listdir()

['Africa_2021',
 'Africa_2021.zip',
 'America_2021',
 'America_2021.zip',
 'Asia_2021',
 'Asia_2021.zip',
 'Europa_2021',
 'Europa_2021.zip',
 'Oceania_2021',
 'Oceania_2021.zip',
 'total_residentes_2021.xlsx']

In [568]:
#Función para obtener datos de residentes
def datos(direccion):
   general=pd.DataFrame()
   for fname in glob.glob(direccion):
      df=pd.read_excel(fname, header=7, usecols="A:B")
      df=df.head(1)
      df['nombre']=fname
      df.columns= df.columns.str.lower()
      df.rename(columns={ df.columns[1]: "residentes" }, inplace = True)
      general=pd.concat([general,df])
   return general

In [569]:
#Directorios de archivos
dir1 = r"D:\ine\America_2021\*.xlsx"
dir2=r"D:\ine\Europa_2021\*.xlsx"
dir3=r"D:\ine\Asia_2021\*.xlsx"
dir4=r"D:\ine\Oceania_2021\*.xlsx"
dir5=r"D:\ine\Africa_2021\*.xlsx"

In [570]:
#Datos
df1=datos(dir1)
df2=datos(dir2)
df3=datos(dir3)
df4=datos(dir4)
df5=datos(dir5)
total=pd.concat([df1,df2,df3,df4,df5])

In [571]:
#limpieza de total
#Obtenemos nombre del país y continente
total["continente"]=total["nombre"].str.split("_",expand=True)[0]
total["pais"]=total["nombre"].str.split("_",expand=True)[1]
total["continente"]=total["continente"].str.replace(r'D:\\ine\\', '', regex=True)
total["pais"]=total["pais"].str.replace(r'2021\\', '', regex=True)
#Llenamos NAs
total["generales"]= total["generales"].fillna("Total de residentes")
#Incorporamos dato de primera generación en Estados Unidos
total.loc[total["pais"] == "Estados Unidos", 'residentes'] = 11750000
#Corregimos dato de Brasil
total.loc[total["pais"] == "Brasil", 'residentes'] = 868
#Dejamos columnas que nos interesan
total.drop(['generación', 'nombre'], axis=1, inplace=True)

In [572]:
total

,generales,residentes,continente,pais
0,Total de residentes,4,America,Antigua y Barbuda
0,Total de residentes,7828,America,Argentina
0,Total de residentes,77,America,Aruba
0,Total de residentes,16,America,Bahamas
0,Total de residentes,3,America,Barbados
...,...,...,...,...
0,Total de residentes,22,Africa,Tanzania
0,Total de residentes,15,Africa,Túnez
0,Total de residentes,11,Africa,Uganda
0,Total de residentes,5,Africa,Zambia


In [573]:
print("Total de mexicanos en el exterior 2021: ",total["residentes"].sum())
print("Total de mexicanos en el exterior 2021 por continente: ",total["residentes"].groupby(total["continente"]).
sum().sort_values(ascending=False))

Total de mexicanos en el exterior 2021:  12145143
Total de mexicanos en el exterior 2021 por continente:  continente
America    11967321
Europa       149418
Asia          17942
Oceania        9449
Africa         1013
Name: residentes, dtype: object


In [574]:
#Exportar a excel
total.to_excel("total_residentes_2021.xlsx", encoding='utf-8', index=False)